In [4]:
# 주요 라이브러리 및 모듈 가져오기
import torch  # 딥러닝 프레임워크 PyTorch
import torch.nn as nn  # 신경망 모듈
import torch.optim as optim  # 최적화 알고리즘
from torch.utils.data import DataLoader, TensorDataset  # 데이터 처리와 로딩 유틸리티
from sklearn.model_selection import train_test_split  # 데이터셋 분할 도구
from sklearn.preprocessing import StandardScaler, LabelEncoder  # 데이터 전처리 도구
import numpy as np  # 수학 연산
import pandas as pd  # 데이터 프레임 처리
from tqdm import tqdm  # 학습 진행 상태 표시
from sklearn.metrics import roc_auc_score, roc_curve  # 모델 성능 평가 지표
import matplotlib.pyplot as plt  # 데이터 시각화
from sklearn.datasets import fetch_openml  # 데이터셋 로드

In [5]:
# 데이터 전처리 함수
def preprocess_dataframe(df):
    df_processed = df.copy()

    for column in df_processed.columns:
        if df_processed[column].dtype == 'category' or df_processed[column].dtype == 'object':
            if pd.api.types.is_categorical_dtype(df_processed[column]):
                df_processed[column] = df_processed[column].cat.add_categories(['Unknown'])
            df_processed[column] = df_processed[column].fillna('Unknown')
            le = LabelEncoder()
            df_processed[column] = le.fit_transform(df_processed[column].astype(str))

        elif df_processed[column].dtype in ['int64', 'float64']:
            df_processed[column] = df_processed[column].fillna(df_processed[column].median())

    return df_processed

In [6]:
# 메인 함수 뜯기

# 데이터 로드 및 전처리
data = pd.read_csv("/content/sample_data/adult.csv",na_values=[' ?']).drop(['fnlwgt'], axis=1).dropna()
df_processed = preprocess_dataframe(data)

<ipython-input-5-150465cc5892>:7: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(df_processed[column]):
<ipython-input-5-150465cc5892>:7: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(df_processed[column]):
<ipython-input-5-150465cc5892>:7: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(df_processed[column]):
<ipython-input-5-150465cc5892>:7: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(df_processed[column]):
<ipython-input-5-150465cc589

In [7]:
# 특성과 타겟, 민감한 특성 분리
sensitive_column = 'gender'  # 예시로 성별 선택
target_column = 'income'   # 예시로 타겟 컬럼 설정
X_data = df_processed.drop([target_column, sensitive_column], axis=1)
y_data = df_processed[target_column]
sensitive_feature = df_processed[sensitive_column]

In [8]:
# 데이터 분할
X_train, X_test, y_train, y_test, sensitive_train, sensitive_test = train_test_split(
    X_data, y_data, sensitive_feature, test_size=0.2, random_state=42, stratify=y_data
)

In [9]:
# 데이터 표준화
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [10]:
# PyTorch 텐서로 변환
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train.values)
y_test = torch.LongTensor(y_test.values)
sensitive_train = torch.LongTensor(sensitive_train.values)
sensitive_test = torch.LongTensor(sensitive_test.values)

In [11]:
# DataLoader 생성
train_dataset = TensorDataset(X_train, y_train, sensitive_train)
test_dataset = TensorDataset(X_test, y_test, sensitive_test)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)

In [12]:
# 통합된 ColumnWiseInteraction 클래스
class ColumnWiseInteraction(nn.Module):
    def __init__(self, input_dim, interaction_dim):
        super().__init__()
        # 명시적인 행렬 곱을 위한 가중치 파라미터
        self.interaction_weights = nn.Parameter(torch.randn(input_dim, interaction_dim))

        # 학습 가능한 비선형 상호작용을 위한 모듈 정의
        self.linear1 = nn.Linear(input_dim, interaction_dim)  # 첫 번째 선형 변환
        self.activation = nn.ReLU()  # 비선형 활성화 함수
        self.linear2 = nn.Linear(interaction_dim, input_dim)  # 두 번째 선형 변환

    def forward(self, x):
        # 명시적인 컬럼 간 상호작용 (matmul 기반)
        matmul_interactions = torch.matmul(x, self.interaction_weights)

        # 학습 가능한 비선형 상호작용
        nonlinear_interactions = self.linear1(x)
        nonlinear_interactions = self.activation(nonlinear_interactions)
        nonlinear_interactions = self.linear2(nonlinear_interactions)

        # 원본 입력, matmul 상호작용, 비선형 상호작용 결합
        return torch.cat([x, matmul_interactions, nonlinear_interactions], dim=1)

In [13]:
# 공정성 지표 계산을 위한 클래스 정의
class FairnessMetrics:
    @staticmethod
    def equal_opportunity_difference(predictions, true_labels, sensitive_features, target_label=1, epsilon=1e-6):
        """
        True Positive Rate(TPR)의 민감한 그룹 간 차이를 계산.
        """
        mask_positive = (true_labels == target_label)  # 실제 레이블이 positive인 경우 필터
        mask_sensitive_0 = (sensitive_features == 0) & mask_positive  # 그룹 0의 positive 필터
        mask_sensitive_1 = (sensitive_features == 1) & mask_positive  # 그룹 1의 positive 필터

        # 각 그룹의 TPR 계산
        tpr_0 = torch.mean(predictions[mask_sensitive_0]) if mask_sensitive_0.sum() > 0 else torch.tensor(0.0).to(predictions.device)
        tpr_1 = torch.mean(predictions[mask_sensitive_1]) if mask_sensitive_1.sum() > 0 else torch.tensor(0.0).to(predictions.device)
        return torch.abs(tpr_0 - tpr_1)  # TPR 차이 반환

    @staticmethod
    def false_positive_rate_difference(predictions, true_labels, sensitive_features, target_label=0, epsilon=1e-6):
        """
        False Positive Rate(FPR)의 민감한 그룹 간 차이를 계산.
        """
        mask_negative = (true_labels == target_label)  # 실제 레이블이 negative인 경우 필터
        mask_sensitive_0 = (sensitive_features == 0) & mask_negative
        mask_sensitive_1 = (sensitive_features == 1) & mask_negative

        # 각 그룹의 FPR 계산
        fpr_0 = torch.mean(predictions[mask_sensitive_0]) if mask_sensitive_0.sum() > 0 else torch.tensor(0.0).to(predictions.device)
        fpr_1 = torch.mean(predictions[mask_sensitive_1]) if mask_sensitive_1.sum() > 0 else torch.tensor(0.0).to(predictions.device)
        return torch.abs(fpr_0 - fpr_1)

    @staticmethod
    def equalized_odds_difference(predictions, true_labels, sensitive_features, target_label=1, epsilon=1e-6):
        """
        Equalized Odds 차이를 계산: EOP와 FPR 차이 중 최대값.
        """
        eop = FairnessMetrics.equal_opportunity_difference(predictions, true_labels, sensitive_features, target_label, epsilon) # True Positive Rate(TPR)의 민감한 그룹 간 차이
        fpr_diff = FairnessMetrics.false_positive_rate_difference(predictions, true_labels, sensitive_features, target_label=0, epsilon=epsilon) # False Positive Rate(FPR)의 민감한 그룹 간 차이
        return torch.max(eop, fpr_diff)  # 두 지표 중 큰 값을 반환

    @staticmethod
    def demographic_parity_difference(predictions, sensitive_features, epsilon=1e-6):
        """
        Positive 예측률의 민감한 그룹 간 차이를 계산.
        """
        mask_sensitive_0 = (sensitive_features == 0)
        mask_sensitive_1 = (sensitive_features == 1)

        rate_0 = torch.mean(predictions[mask_sensitive_0]) if mask_sensitive_0.sum() > 0 else torch.tensor(0.0).to(predictions.device)
        rate_1 = torch.mean(predictions[mask_sensitive_1]) if mask_sensitive_1.sum() > 0 else torch.tensor(0.0).to(predictions.device)
        return torch.abs(rate_0 - rate_1)  # 차이 반환

In [14]:
# 공정성을 고려한 Transformer 분류기 정의
class FairnessAwareTransformerClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_heads, num_classes, interaction_dim=32):
        super().__init__()
        self.column_interaction = ColumnWiseInteraction(input_dim, interaction_dim)
        self.input_projection = nn.Linear(1, hidden_dim)  # 각 특성을 임베딩
        self.hidden_dim = hidden_dim  # Hidden dimension 저장
        self.num_heads = num_heads  # Transformer 헤드 수
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=hidden_dim, nhead=num_heads, dim_feedforward=hidden_dim * 4, batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=2)
        self.classifier = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim, num_classes)
        )

    def forward(self, x):
        # Column-wise interaction 적용
        x = self.column_interaction(x)
        seq_len = x.size(1)  # 입력의 sequence 길이 (특성 수)

        # Input projection 및 차원 확장
        x = x.unsqueeze(-1)  # (batch_size, seq_len, 1)
        x = self.input_projection(x)  # (batch_size, seq_len, hidden_dim)

        # 동적으로 위치 인코딩 생성
        positional_encoding = torch.randn(1, seq_len, self.hidden_dim, device=x.device)
        x = x + positional_encoding

        # Transformer Encoder 적용
        x = self.transformer_encoder(x)

        # Sequence의 평균을 사용하여 분류 입력 생성
        x = x.mean(dim=1)
        return self.classifier(x)

In [15]:
# 학습 함수
def train_model(model, train_loader, criterion, optimizer, device, lambda_fairness=0.01):
    model.train()
    total_loss = 0
    total_fairness_loss = 0
    total_eop = 0
    total_eo = 0
    total_dp = 0
    total_batches = 0

    for batch_x, batch_y, batch_sensitive in train_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        batch_sensitive = batch_sensitive.to(device)

        optimizer.zero_grad()

        # 예측 및 분류 손실
        outputs = model(batch_x)
        classification_loss = criterion(outputs, batch_y)

        # 공정성 손실 계산
        probabilities = torch.softmax(outputs, dim=1)[:, 1]  # Probability of class 1

        # To avoid NaNs, ensure that probabilities are between 0 and 1
        probabilities = torch.clamp(probabilities, min=0.0, max=1.0)

        # 공정성 지표 계산
        eop = FairnessMetrics.equal_opportunity_difference(probabilities, batch_y, batch_sensitive, target_label=1)
        eo = FairnessMetrics.equalized_odds_difference(probabilities, batch_y, batch_sensitive, target_label=1, epsilon=1e-6)
        dp = FairnessMetrics.demographic_parity_difference(probabilities, batch_sensitive)

        # 공정성 손실 합산 (가중치를 조정할 수 있습니다)
        fairness_loss = eop + eo + dp

        # 전체 손실 결합
        total_loss_batch = classification_loss + lambda_fairness * fairness_loss
        total_loss_batch.backward()
        optimizer.step()

        # 손실 및 공정성 지표 기록
        total_loss += classification_loss.item()
        total_fairness_loss += fairness_loss.item()
        total_eop += eop.item()
        total_eo += eo.item()
        total_dp += dp.item()
        total_batches += 1

    avg_loss = total_loss / total_batches
    avg_fairness_loss = total_fairness_loss / total_batches
    avg_eop = total_eop / total_batches
    avg_eo = total_eo / total_batches
    avg_dp = total_dp / total_batches

    return avg_loss, avg_fairness_loss, avg_eop, avg_eo, avg_dp

In [16]:
# 평가 함수
def evaluate_model(model, test_loader, device):
    model.eval()
    correct = 0
    total = 0
    all_predictions = []
    all_labels = []
    all_sensitive = []

    with torch.no_grad():
        for batch_x, batch_y, batch_sensitive in test_loader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            outputs = model(batch_x)
            _, predicted = torch.max(outputs.data, 1)
            total += batch_y.size(0)
            correct += (predicted == batch_y).sum().item()

            # 공정성 지표 계산을 위한 데이터 수집
            probabilities = torch.softmax(outputs, dim=1)[:, 1]
            all_predictions.append(probabilities.cpu())
            all_labels.append(batch_y.cpu())
            all_sensitive.append(batch_sensitive.cpu())

    accuracy = 100 * correct / total

    # 모든 배치를 하나로 합침
    all_predictions = torch.cat(all_predictions)
    all_labels = torch.cat(all_labels)
    all_sensitive = torch.cat(all_sensitive)

    # 공정성 지표 계산
    eop = FairnessMetrics.equal_opportunity_difference(all_predictions, all_labels, all_sensitive, target_label=1).item()
    eo = FairnessMetrics.equalized_odds_difference(all_predictions, all_labels, all_sensitive, target_label=1, epsilon=1e-6).item()
    dp = FairnessMetrics.demographic_parity_difference(all_predictions, all_sensitive).item()

    # AUROC 계산
    auroc = roc_auc_score(all_labels.numpy(), all_predictions.numpy())

    return accuracy, eop, eo, dp, auroc

In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 모델 초기화
input_dim = X_train.shape[1]
hidden_dim = 128
num_heads = 4
num_classes = 2

model = FairnessAwareTransformerClassifier(input_dim, hidden_dim, num_heads, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10  # 충분한 학습을 위해 epoch 수 증가

# 기록을 위한 리스트 초기화
train_fairness_history = {
    'loss': [],
    'fairness_loss': [],
    'EOP': [],
    'EO': [],
    'DP': []
}
test_fairness_history = {
    'accuracy': [],
    'EOP': [],
    'EO': [],
    'DP': []
}

for epoch in tqdm(range(num_epochs), desc="Training"):
    train_loss, train_fairness_loss, train_eop, train_eo, train_dp = train_model(
        model, train_loader, criterion, optimizer,
        device, lambda_fairness=0.1
    )
    test_acc, test_eop, test_eo, test_dp, test_auroc = evaluate_model(model, test_loader, device)

    # 기록 저장
    train_fairness_history['loss'].append(train_loss)
    train_fairness_history['fairness_loss'].append(train_fairness_loss)
    train_fairness_history['EOP'].append(train_eop)
    train_fairness_history['EO'].append(train_eo)
    train_fairness_history['DP'].append(train_dp)

    test_fairness_history['accuracy'].append(test_acc)
    test_fairness_history['EOP'].append(test_eop)
    test_fairness_history['EO'].append(test_eo)
    test_fairness_history['DP'].append(test_dp)

    print(f'Epoch [{epoch+1}/{num_epochs}], '
          f'Loss: {train_loss:.4f}, '
          f'Fairness Loss: {train_fairness_loss:.4f}, '
          f'EOP: {train_eop:.4f}, '
          f'EO: {train_eo:.4f}, '
          f'DP: {train_dp:.4f}, '
          f'Test Accuracy: {test_acc:.2f}%, '
          f'Test EOP: {test_eop:.4f}, '
          f'Test EO: {test_eo:.4f}, '
          f'Test DP: {test_dp:.4f}',
          f'Test AUROC: {test_auroc:.4f}')


Training:  10%|█         | 1/10 [00:09<01:27,  9.67s/it]

Epoch [1/10], Loss: 0.4326, Fairness Loss: 0.3253, EOP: 0.1210, EO: 0.1305, DP: 0.0738, Test Accuracy: 82.15%, Test EOP: 0.0367, Test EO: 0.0367, Test DP: 0.0777 Test AUROC: 0.8584


Training:  20%|██        | 2/10 [00:16<01:04,  8.09s/it]

Epoch [2/10], Loss: 0.3795, Fairness Loss: 0.4184, EOP: 0.1494, EO: 0.1622, DP: 0.1068, Test Accuracy: 83.65%, Test EOP: 0.0275, Test EO: 0.0596, Test DP: 0.1149 Test AUROC: 0.8835


Training:  30%|███       | 3/10 [00:23<00:53,  7.71s/it]

...

Epoch [8/10], Loss: 0.3487, Fairness Loss: 0.4795, EOP: 0.1635, EO: 0.1812, DP: 0.1347, Test Accuracy: 84.02%, Test EOP: 0.0271, Test EO: 0.0777, Test DP: 0.1400 Test AUROC: 0.8936


Training:  90%|█████████ | 9/10 [01:07<00:07,  7.27s/it]

Epoch [9/10], Loss: 0.3462, Fairness Loss: 0.4726, EOP: 0.1628, EO: 0.1796, DP: 0.1301, Test Accuracy: 84.14%, Test EOP: 0.0313, Test EO: 0.0720, Test DP: 0.1332 Test AUROC: 0.8926


Training: 100%|██████████| 10/10 [01:14<00:00,  7.48s/it]

Epoch [10/10], Loss: 0.3464, Fairness Loss: 0.4911, EOP: 0.1705, EO: 0.1875, DP: 0.1330, Test Accuracy: 84.52%, Test EOP: 0.0224, Test EO: 0.0670, Test DP: 0.1315 Test AUROC: 0.8917


# 1) EOP: 0.1 / EO: 0.45 / DP: 0.45

In [18]:
# 학습 함수
def train_model(model, train_loader, criterion, optimizer, device, lambda_fairness=0.01):
    model.train()
    total_loss = 0
    total_fairness_loss = 0
    total_eop = 0
    total_eo = 0
    total_dp = 0
    total_batches = 0

    for batch_x, batch_y, batch_sensitive in train_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        batch_sensitive = batch_sensitive.to(device)

        optimizer.zero_grad()

        # 예측 및 분류 손실
        outputs = model(batch_x)
        classification_loss = criterion(outputs, batch_y)

        # 공정성 손실 계산
        probabilities = torch.softmax(outputs, dim=1)[:, 1]  # Probability of class 1

        # To avoid NaNs, ensure that probabilities are between 0 and 1
        probabilities = torch.clamp(probabilities, min=0.0, max=1.0)

        # 공정성 지표 계산
        eop = FairnessMetrics.equal_opportunity_difference(probabilities, batch_y, batch_sensitive, target_label=1)
        eo = FairnessMetrics.equalized_odds_difference(probabilities, batch_y, batch_sensitive, target_label=1, epsilon=1e-6)
        dp = FairnessMetrics.demographic_parity_difference(probabilities, batch_sensitive)

        # 공정성 가중치 조정
        a = 0.1
        b = 0.45
        c = 0.45


        # 공정성 손실 합산
        fairness_loss = (a * eop) + (b * eo) +  (c * dp)

        # 전체 손실 결합
        total_loss_batch = classification_loss + lambda_fairness * fairness_loss
        total_loss_batch.backward()
        optimizer.step()

        # 손실 및 공정성 지표 기록
        total_loss += classification_loss.item()
        total_fairness_loss += fairness_loss.item()
        total_eop += eop.item()
        total_eo += eo.item()
        total_dp += dp.item()
        total_batches += 1

    avg_loss = total_loss / total_batches
    avg_fairness_loss = total_fairness_loss / total_batches
    avg_eop = total_eop / total_batches
    avg_eo = total_eo / total_batches
    avg_dp = total_dp / total_batches

    return avg_loss, avg_fairness_loss, avg_eop, avg_eo, avg_dp

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 모델 초기화
input_dim = X_train.shape[1]
hidden_dim = 128
num_heads = 4
num_classes = 2

model = FairnessAwareTransformerClassifier(input_dim, hidden_dim, num_heads, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10  # 충분한 학습을 위해 epoch 수 증가

# 기록을 위한 리스트 초기화
train_fairness_history = {
    'loss': [],
    'fairness_loss': [],
    'EOP': [],
    'EO': [],
    'DP': []
}
test_fairness_history = {
    'accuracy': [],
    'EOP': [],
    'EO': [],
    'DP': []
}

for epoch in tqdm(range(num_epochs), desc="Training"):
    train_loss, train_fairness_loss, train_eop, train_eo, train_dp = train_model(
        model, train_loader, criterion, optimizer,
        device, lambda_fairness=0.1
    )
    test_acc, test_eop, test_eo, test_dp, test_auroc = evaluate_model(model, test_loader, device)

    # 기록 저장
    train_fairness_history['loss'].append(train_loss)
    train_fairness_history['fairness_loss'].append(train_fairness_loss)
    train_fairness_history['EOP'].append(train_eop)
    train_fairness_history['EO'].append(train_eo)
    train_fairness_history['DP'].append(train_dp)

    test_fairness_history['accuracy'].append(test_acc)
    test_fairness_history['EOP'].append(test_eop)
    test_fairness_history['EO'].append(test_eo)
    test_fairness_history['DP'].append(test_dp)

    print(f'Epoch [{epoch+1}/{num_epochs}], '
          f'Loss: {train_loss:.4f}, '
          f'Fairness Loss: {train_fairness_loss:.4f}, '
          f'EOP: {train_eop:.4f}, '
          f'EO: {train_eo:.4f}, '
          f'DP: {train_dp:.4f}, '
          f'Test Accuracy: {test_acc:.2f}%, '
          f'Test EOP: {test_eop:.4f}, '
          f'Test EO: {test_eo:.4f}, '
          f'Test DP: {test_dp:.4f}',
          f'Test AUROC: {test_auroc:.4f}')


Training:  10%|█         | 1/10 [00:07<01:08,  7.62s/it]

Epoch [1/10], Loss: 0.4194, Fairness Loss: 0.1604, EOP: 0.1946, EO: 0.2021, DP: 0.1111, Test Accuracy: 82.38%, Test EOP: 0.0941, Test EO: 0.0941, Test DP: 0.1147 Test AUROC: 0.8657


Training:  20%|██        | 2/10 [00:14<00:58,  7.32s/it]

Epoch [2/10], Loss: 0.3673, Fairness Loss: 0.1885, EOP: 0.2132, EO: 0.2261, DP: 0.1454, Test Accuracy: 84.38%, Test EOP: 0.1050, Test EO: 0.1050, Test DP: 0.1636 Test AUROC: 0.8895


Training:  30%|███       | 3/10 [00:22<00:52,  7.44s/it]

...

Epoch [8/10], Loss: 0.3353, Fairness Loss: 0.1926, EOP: 0.2048, EO: 0.2209, DP: 0.1616, Test Accuracy: 84.88%, Test EOP: 0.0760, Test EO: 0.0872, Test DP: 0.1585 Test AUROC: 0.8965


Training:  90%|█████████ | 9/10 [01:05<00:07,  7.22s/it]

Epoch [9/10], Loss: 0.3387, Fairness Loss: 0.1901, EOP: 0.2065, EO: 0.2218, DP: 0.1547, Test Accuracy: 84.88%, Test EOP: 0.0586, Test EO: 0.0837, Test DP: 0.1552 Test AUROC: 0.8952


Training: 100%|██████████| 10/10 [01:13<00:00,  7.33s/it]

Epoch [10/10], Loss: 0.3396, Fairness Loss: 0.1909, EOP: 0.2058, EO: 0.2235, DP: 0.1551, Test Accuracy: 84.68%, Test EOP: 0.0462, Test EO: 0.0785, Test DP: 0.1486 Test AUROC: 0.8959


# 2) EOP: 0.5 / EO: 0.25 / DP: 0.25

In [28]:
# 학습 함수
def train_model(model, train_loader, criterion, optimizer, device, lambda_fairness=0.01):
    model.train()
    total_loss = 0
    total_fairness_loss = 0
    total_eop = 0
    total_eo = 0
    total_dp = 0
    total_batches = 0

    for batch_x, batch_y, batch_sensitive in train_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        batch_sensitive = batch_sensitive.to(device)

        optimizer.zero_grad()

        # 예측 및 분류 손실
        outputs = model(batch_x)
        classification_loss = criterion(outputs, batch_y)

        # 공정성 손실 계산
        probabilities = torch.softmax(outputs, dim=1)[:, 1]  # Probability of class 1

        # To avoid NaNs, ensure that probabilities are between 0 and 1
        probabilities = torch.clamp(probabilities, min=0.0, max=1.0)

        # 공정성 지표 계산
        eop = FairnessMetrics.equal_opportunity_difference(probabilities, batch_y, batch_sensitive, target_label=1)
        eo = FairnessMetrics.equalized_odds_difference(probabilities, batch_y, batch_sensitive, target_label=1, epsilon=1e-6)
        dp = FairnessMetrics.demographic_parity_difference(probabilities, batch_sensitive)

        # 공정성 가중치 조정
        a = 0.5
        b = 0.25
        c = 0.25


        # 공정성 손실 합산
        fairness_loss = (a * eop) + (b * eo) +  (c * dp)

        # 전체 손실 결합
        total_loss_batch = classification_loss + lambda_fairness * fairness_loss
        total_loss_batch.backward()
        optimizer.step()

        # 손실 및 공정성 지표 기록
        total_loss += classification_loss.item()
        total_fairness_loss += fairness_loss.item()
        total_eop += eop.item()
        total_eo += eo.item()
        total_dp += dp.item()
        total_batches += 1

    avg_loss = total_loss / total_batches
    avg_fairness_loss = total_fairness_loss / total_batches
    avg_eop = total_eop / total_batches
    avg_eo = total_eo / total_batches
    avg_dp = total_dp / total_batches

    return avg_loss, avg_fairness_loss, avg_eop, avg_eo, avg_dp

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 모델 초기화
input_dim = X_train.shape[1]
hidden_dim = 128
num_heads = 4
num_classes = 2

model = FairnessAwareTransformerClassifier(input_dim, hidden_dim, num_heads, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10  # 충분한 학습을 위해 epoch 수 증가

# 기록을 위한 리스트 초기화
train_fairness_history = {
    'loss': [],
    'fairness_loss': [],
    'EOP': [],
    'EO': [],
    'DP': []
}
test_fairness_history = {
    'accuracy': [],
    'EOP': [],
    'EO': [],
    'DP': []
}

for epoch in tqdm(range(num_epochs), desc="Training"):
    train_loss, train_fairness_loss, train_eop, train_eo, train_dp = train_model(
        model, train_loader, criterion, optimizer,
        device, lambda_fairness=0.1
    )
    test_acc, test_eop, test_eo, test_dp, test_auroc = evaluate_model(model, test_loader, device)

    # 기록 저장
    train_fairness_history['loss'].append(train_loss)
    train_fairness_history['fairness_loss'].append(train_fairness_loss)
    train_fairness_history['EOP'].append(train_eop)
    train_fairness_history['EO'].append(train_eo)
    train_fairness_history['DP'].append(train_dp)

    test_fairness_history['accuracy'].append(test_acc)
    test_fairness_history['EOP'].append(test_eop)
    test_fairness_history['EO'].append(test_eo)
    test_fairness_history['DP'].append(test_dp)

    print(f'Epoch [{epoch+1}/{num_epochs}], '
          f'Loss: {train_loss:.4f}, '
          f'Fairness Loss: {train_fairness_loss:.4f}, '
          f'EOP: {train_eop:.4f}, '
          f'EO: {train_eo:.4f}, '
          f'DP: {train_dp:.4f}, '
          f'Test Accuracy: {test_acc:.2f}%, '
          f'Test EOP: {test_eop:.4f}, '
          f'Test EO: {test_eo:.4f}, '
          f'Test DP: {test_dp:.4f}',
          f'Test AUROC: {test_auroc:.4f}')

Training:  10%|█         | 1/10 [00:07<01:03,  7.08s/it]

Epoch [1/10], Loss: 0.4562, Fairness Loss: 0.1344, EOP: 0.1498, EO: 0.1566, DP: 0.0815, Test Accuracy: 79.73%, Test EOP: 0.0976, Test EO: 0.0976, Test DP: 0.1070 Test AUROC: 0.8447


Training:  20%|██        | 2/10 [00:14<00:57,  7.22s/it]

Epoch [2/10], Loss: 0.4016, Fairness Loss: 0.1844, EOP: 0.2001, EO: 0.2094, DP: 0.1282, Test Accuracy: 83.13%, Test EOP: 0.1036, Test EO: 0.1036, Test DP: 0.1474 Test AUROC: 0.8666


Training:  30%|███       | 3/10 [00:21<00:51,  7.34s/it]

...

Epoch [8/10], Loss: 0.3516, Fairness Loss: 0.1930, EOP: 0.2007, EO: 0.2177, DP: 0.1528, Test Accuracy: 84.38%, Test EOP: 0.0562, Test EO: 0.0938, Test DP: 0.1623 Test AUROC: 0.8917


Training:  90%|█████████ | 9/10 [01:05<00:07,  7.28s/it]

Epoch [9/10], Loss: 0.3506, Fairness Loss: 0.1977, EOP: 0.2035, EO: 0.2232, DP: 0.1606, Test Accuracy: 84.14%, Test EOP: 0.0378, Test EO: 0.0753, Test DP: 0.1399 Test AUROC: 0.8895


Training: 100%|██████████| 10/10 [01:12<00:00,  7.30s/it]

Epoch [10/10], Loss: 0.3492, Fairness Loss: 0.1991, EOP: 0.2074, EO: 0.2234, DP: 0.1583, Test Accuracy: 84.07%, Test EOP: 0.0347, Test EO: 0.1092, Test DP: 0.1762 Test AUROC: 0.8898


# 3) EOP: 0.9 / EO: 0.05 / DP: 0.05

In [20]:
# 학습 함수
def train_model(model, train_loader, criterion, optimizer, device, lambda_fairness=0.01):
    model.train()
    total_loss = 0
    total_fairness_loss = 0
    total_eop = 0
    total_eo = 0
    total_dp = 0
    total_batches = 0

    for batch_x, batch_y, batch_sensitive in train_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        batch_sensitive = batch_sensitive.to(device)

        optimizer.zero_grad()

        # 예측 및 분류 손실
        outputs = model(batch_x)
        classification_loss = criterion(outputs, batch_y)

        # 공정성 손실 계산
        probabilities = torch.softmax(outputs, dim=1)[:, 1]  # Probability of class 1

        # To avoid NaNs, ensure that probabilities are between 0 and 1
        probabilities = torch.clamp(probabilities, min=0.0, max=1.0)

        # 공정성 지표 계산
        eop = FairnessMetrics.equal_opportunity_difference(probabilities, batch_y, batch_sensitive, target_label=1)
        eo = FairnessMetrics.equalized_odds_difference(probabilities, batch_y, batch_sensitive, target_label=1, epsilon=1e-6)
        dp = FairnessMetrics.demographic_parity_difference(probabilities, batch_sensitive)

        # 공정성 가중치 조정
        a = 0.9
        b = 0.05
        c = 0.05


        # 공정성 손실 합산
        fairness_loss = (a * eop) + (b * eo) +  (c * dp)

        # 전체 손실 결합
        total_loss_batch = classification_loss + lambda_fairness * fairness_loss
        total_loss_batch.backward()
        optimizer.step()

        # 손실 및 공정성 지표 기록
        total_loss += classification_loss.item()
        total_fairness_loss += fairness_loss.item()
        total_eop += eop.item()
        total_eo += eo.item()
        total_dp += dp.item()
        total_batches += 1

    avg_loss = total_loss / total_batches
    avg_fairness_loss = total_fairness_loss / total_batches
    avg_eop = total_eop / total_batches
    avg_eo = total_eo / total_batches
    avg_dp = total_dp / total_batches

    return avg_loss, avg_fairness_loss, avg_eop, avg_eo, avg_dp

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 모델 초기화
input_dim = X_train.shape[1]
hidden_dim = 128
num_heads = 4
num_classes = 2

model = FairnessAwareTransformerClassifier(input_dim, hidden_dim, num_heads, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10  # 충분한 학습을 위해 epoch 수 증가

# 기록을 위한 리스트 초기화
train_fairness_history = {
    'loss': [],
    'fairness_loss': [],
    'EOP': [],
    'EO': [],
    'DP': []
}
test_fairness_history = {
    'accuracy': [],
    'EOP': [],
    'EO': [],
    'DP': []
}

for epoch in tqdm(range(num_epochs), desc="Training"):
    train_loss, train_fairness_loss, train_eop, train_eo, train_dp = train_model(
        model, train_loader, criterion, optimizer,
        device, lambda_fairness=0.1
    )
    test_acc, test_eop, test_eo, test_dp, test_auroc = evaluate_model(model, test_loader, device)

    # 기록 저장
    train_fairness_history['loss'].append(train_loss)
    train_fairness_history['fairness_loss'].append(train_fairness_loss)
    train_fairness_history['EOP'].append(train_eop)
    train_fairness_history['EO'].append(train_eo)
    train_fairness_history['DP'].append(train_dp)

    test_fairness_history['accuracy'].append(test_acc)
    test_fairness_history['EOP'].append(test_eop)
    test_fairness_history['EO'].append(test_eo)
    test_fairness_history['DP'].append(test_dp)

    print(f'Epoch [{epoch+1}/{num_epochs}], '
          f'Loss: {train_loss:.4f}, '
          f'Fairness Loss: {train_fairness_loss:.4f}, '
          f'EOP: {train_eop:.4f}, '
          f'EO: {train_eo:.4f}, '
          f'DP: {train_dp:.4f}, '
          f'Test Accuracy: {test_acc:.2f}%, '
          f'Test EOP: {test_eop:.4f}, '
          f'Test EO: {test_eo:.4f}, '
          f'Test DP: {test_dp:.4f}',
          f'Test AUROC: {test_auroc:.4f}')

Training:  10%|█         | 1/10 [00:06<01:02,  6.91s/it]

Epoch [1/10], Loss: 0.4279, Fairness Loss: 0.1901, EOP: 0.1938, EO: 0.2018, DP: 0.1117, Test Accuracy: 82.09%, Test EOP: 0.0888, Test EO: 0.0888, Test DP: 0.1127 Test AUROC: 0.8485


Training:  20%|██        | 2/10 [00:14<00:58,  7.34s/it]

Epoch [2/10], Loss: 0.3925, Fairness Loss: 0.1945, EOP: 0.1973, EO: 0.2079, DP: 0.1315, Test Accuracy: 81.99%, Test EOP: 0.1048, Test EO: 0.1048, Test DP: 0.1219 Test AUROC: 0.8617


Training:  30%|███       | 3/10 [00:21<00:50,  7.16s/it]

...

Epoch [8/10], Loss: 0.5090, Fairness Loss: 0.0941, EOP: 0.0964, EO: 0.1014, DP: 0.0449, Test Accuracy: 76.13%, Test EOP: 0.0191, Test EO: 0.0191, Test DP: 0.0110 Test AUROC: 0.4761


Training:  90%|█████████ | 9/10 [01:07<00:07,  7.67s/it]

Epoch [9/10], Loss: 0.5029, Fairness Loss: 0.0974, EOP: 0.0998, EO: 0.1047, DP: 0.0475, Test Accuracy: 80.63%, Test EOP: 0.0816, Test EO: 0.0816, Test DP: 0.0827 Test AUROC: 0.7934


Training: 100%|██████████| 10/10 [01:15<00:00,  7.53s/it]

Epoch [10/10], Loss: 0.4213, Fairness Loss: 0.2116, EOP: 0.2164, EO: 0.2220, DP: 0.1160, Test Accuracy: 76.14%, Test EOP: 0.0292, Test EO: 0.0404, Test DP: 0.0588 Test AUROC: 0.8682


# 4) EOP:0.4 / OP:0.2 / DP:0.4

In [21]:
# 학습 함수
def train_model(model, train_loader, criterion, optimizer, device, lambda_fairness=0.01):
    model.train()
    total_loss = 0
    total_fairness_loss = 0
    total_eop = 0
    total_eo = 0
    total_dp = 0
    total_batches = 0

    for batch_x, batch_y, batch_sensitive in train_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        batch_sensitive = batch_sensitive.to(device)

        optimizer.zero_grad()

        # 예측 및 분류 손실
        outputs = model(batch_x)
        classification_loss = criterion(outputs, batch_y)

        # 공정성 손실 계산
        probabilities = torch.softmax(outputs, dim=1)[:, 1]  # Probability of class 1

        # To avoid NaNs, ensure that probabilities are between 0 and 1
        probabilities = torch.clamp(probabilities, min=0.0, max=1.0)

        # 공정성 지표 계산
        eop = FairnessMetrics.equal_opportunity_difference(probabilities, batch_y, batch_sensitive, target_label=1)
        eo = FairnessMetrics.equalized_odds_difference(probabilities, batch_y, batch_sensitive, target_label=1, epsilon=1e-6)
        dp = FairnessMetrics.demographic_parity_difference(probabilities, batch_sensitive)

        # 공정성 가중치 조정
        a = 0.4
        b = 0.2
        c = 0.4


        # 공정성 손실 합산
        fairness_loss = (a * eop) + (b * eo) +  (c * dp)

        # 전체 손실 결합
        total_loss_batch = classification_loss + lambda_fairness * fairness_loss
        total_loss_batch.backward()
        optimizer.step()

        # 손실 및 공정성 지표 기록
        total_loss += classification_loss.item()
        total_fairness_loss += fairness_loss.item()
        total_eop += eop.item()
        total_eo += eo.item()
        total_dp += dp.item()
        total_batches += 1

    avg_loss = total_loss / total_batches
    avg_fairness_loss = total_fairness_loss / total_batches
    avg_eop = total_eop / total_batches
    avg_eo = total_eo / total_batches
    avg_dp = total_dp / total_batches

    return avg_loss, avg_fairness_loss, avg_eop, avg_eo, avg_dp

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 모델 초기화
input_dim = X_train.shape[1]
hidden_dim = 128
num_heads = 4
num_classes = 2

model = FairnessAwareTransformerClassifier(input_dim, hidden_dim, num_heads, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10  # 충분한 학습을 위해 epoch 수 증가

# 기록을 위한 리스트 초기화
train_fairness_history = {
    'loss': [],
    'fairness_loss': [],
    'EOP': [],
    'EO': [],
    'DP': []
}
test_fairness_history = {
    'accuracy': [],
    'EOP': [],
    'EO': [],
    'DP': []
}

for epoch in tqdm(range(num_epochs), desc="Training"):
    train_loss, train_fairness_loss, train_eop, train_eo, train_dp = train_model(
        model, train_loader, criterion, optimizer,
        device, lambda_fairness=0.1
    )
    test_acc, test_eop, test_eo, test_dp, test_auroc = evaluate_model(model, test_loader, device)

    # 기록 저장
    train_fairness_history['loss'].append(train_loss)
    train_fairness_history['fairness_loss'].append(train_fairness_loss)
    train_fairness_history['EOP'].append(train_eop)
    train_fairness_history['EO'].append(train_eo)
    train_fairness_history['DP'].append(train_dp)

    test_fairness_history['accuracy'].append(test_acc)
    test_fairness_history['EOP'].append(test_eop)
    test_fairness_history['EO'].append(test_eo)
    test_fairness_history['DP'].append(test_dp)

    print(f'Epoch [{epoch+1}/{num_epochs}], '
          f'Loss: {train_loss:.4f}, '
          f'Fairness Loss: {train_fairness_loss:.4f}, '
          f'EOP: {train_eop:.4f}, '
          f'EO: {train_eo:.4f}, '
          f'DP: {train_dp:.4f}, '
          f'Test Accuracy: {test_acc:.2f}%, '
          f'Test EOP: {test_eop:.4f}, '
          f'Test EO: {test_eo:.4f}, '
          f'Test DP: {test_dp:.4f}',
          f'Test AUROC: {test_auroc:.4f}')

Training:  10%|█         | 1/10 [00:07<01:04,  7.18s/it]

Epoch [1/10], Loss: 0.4303, Fairness Loss: 0.1521, EOP: 0.1681, EO: 0.1807, DP: 0.1219, Test Accuracy: 82.41%, Test EOP: 0.0831, Test EO: 0.0956, Test DP: 0.1622 Test AUROC: 0.8735


Training:  20%|██        | 2/10 [00:14<00:59,  7.42s/it]

Epoch [2/10], Loss: 0.3650, Fairness Loss: 0.1800, EOP: 0.1915, EO: 0.2107, DP: 0.1531, Test Accuracy: 83.90%, Test EOP: 0.0408, Test EO: 0.0905, Test DP: 0.1572 Test AUROC: 0.8872


Training:  30%|███       | 3/10 [00:21<00:50,  7.18s/it]

...

Epoch [8/10], Loss: 0.3392, Fairness Loss: 0.1950, EOP: 0.2109, EO: 0.2293, DP: 0.1619, Test Accuracy: 84.35%, Test EOP: 0.0146, Test EO: 0.0956, Test DP: 0.1651 Test AUROC: 0.8928


Training:  90%|█████████ | 9/10 [01:05<00:07,  7.37s/it]

Epoch [9/10], Loss: 0.3393, Fairness Loss: 0.1884, EOP: 0.1998, EO: 0.2183, DP: 0.1621, Test Accuracy: 84.36%, Test EOP: 0.0267, Test EO: 0.0828, Test DP: 0.1489 Test AUROC: 0.8929


Training: 100%|██████████| 10/10 [01:12<00:00,  7.28s/it]

Epoch [10/10], Loss: 0.3392, Fairness Loss: 0.1970, EOP: 0.2091, EO: 0.2301, DP: 0.1684, Test Accuracy: 84.17%, Test EOP: 0.0460, Test EO: 0.1124, Test DP: 0.1831 Test AUROC: 0.8934


# 5) EOP:0.45 / OP:0.1 / DP:0.45

In [22]:
# 학습 함수
def train_model(model, train_loader, criterion, optimizer, device, lambda_fairness=0.01):
    model.train()
    total_loss = 0
    total_fairness_loss = 0
    total_eop = 0
    total_eo = 0
    total_dp = 0
    total_batches = 0

    for batch_x, batch_y, batch_sensitive in train_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        batch_sensitive = batch_sensitive.to(device)

        optimizer.zero_grad()

        # 예측 및 분류 손실
        outputs = model(batch_x)
        classification_loss = criterion(outputs, batch_y)

        # 공정성 손실 계산
        probabilities = torch.softmax(outputs, dim=1)[:, 1]  # Probability of class 1

        # To avoid NaNs, ensure that probabilities are between 0 and 1
        probabilities = torch.clamp(probabilities, min=0.0, max=1.0)

        # 공정성 지표 계산
        eop = FairnessMetrics.equal_opportunity_difference(probabilities, batch_y, batch_sensitive, target_label=1)
        eo = FairnessMetrics.equalized_odds_difference(probabilities, batch_y, batch_sensitive, target_label=1, epsilon=1e-6)
        dp = FairnessMetrics.demographic_parity_difference(probabilities, batch_sensitive)

        # 공정성 가중치 조정
        a = 0.45
        b = 0.1
        c = 0.45


        # 공정성 손실 합산
        fairness_loss = (a * eop) + (b * eo) +  (c * dp)

        # 전체 손실 결합
        total_loss_batch = classification_loss + lambda_fairness * fairness_loss
        total_loss_batch.backward()
        optimizer.step()

        # 손실 및 공정성 지표 기록
        total_loss += classification_loss.item()
        total_fairness_loss += fairness_loss.item()
        total_eop += eop.item()
        total_eo += eo.item()
        total_dp += dp.item()
        total_batches += 1

    avg_loss = total_loss / total_batches
    avg_fairness_loss = total_fairness_loss / total_batches
    avg_eop = total_eop / total_batches
    avg_eo = total_eo / total_batches
    avg_dp = total_dp / total_batches

    return avg_loss, avg_fairness_loss, avg_eop, avg_eo, avg_dp

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 모델 초기화
input_dim = X_train.shape[1]
hidden_dim = 128
num_heads = 4
num_classes = 2

model = FairnessAwareTransformerClassifier(input_dim, hidden_dim, num_heads, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10  # 충분한 학습을 위해 epoch 수 증가

# 기록을 위한 리스트 초기화
train_fairness_history = {
    'loss': [],
    'fairness_loss': [],
    'EOP': [],
    'EO': [],
    'DP': []
}
test_fairness_history = {
    'accuracy': [],
    'EOP': [],
    'EO': [],
    'DP': []
}

for epoch in tqdm(range(num_epochs), desc="Training"):
    train_loss, train_fairness_loss, train_eop, train_eo, train_dp = train_model(
        model, train_loader, criterion, optimizer,
        device, lambda_fairness=0.1
    )
    test_acc, test_eop, test_eo, test_dp, test_auroc = evaluate_model(model, test_loader, device)

    # 기록 저장
    train_fairness_history['loss'].append(train_loss)
    train_fairness_history['fairness_loss'].append(train_fairness_loss)
    train_fairness_history['EOP'].append(train_eop)
    train_fairness_history['EO'].append(train_eo)
    train_fairness_history['DP'].append(train_dp)

    test_fairness_history['accuracy'].append(test_acc)
    test_fairness_history['EOP'].append(test_eop)
    test_fairness_history['EO'].append(test_eo)
    test_fairness_history['DP'].append(test_dp)

    print(f'Epoch [{epoch+1}/{num_epochs}], '
          f'Loss: {train_loss:.4f}, '
          f'Fairness Loss: {train_fairness_loss:.4f}, '
          f'EOP: {train_eop:.4f}, '
          f'EO: {train_eo:.4f}, '
          f'DP: {train_dp:.4f}, '
          f'Test Accuracy: {test_acc:.2f}%, '
          f'Test EOP: {test_eop:.4f}, '
          f'Test EO: {test_eo:.4f}, '
          f'Test DP: {test_dp:.4f}',
          f'Test AUROC: {test_auroc:.4f}')

Training:  10%|█         | 1/10 [00:07<01:08,  7.64s/it]

Epoch [1/10], Loss: 0.4489, Fairness Loss: 0.1508, EOP: 0.1849, EO: 0.1925, DP: 0.1074, Test Accuracy: 79.56%, Test EOP: 0.1401, Test EO: 0.1401, Test DP: 0.1411 Test AUROC: 0.8331


Training:  20%|██        | 2/10 [00:14<00:58,  7.30s/it]

Epoch [2/10], Loss: 0.4246, Fairness Loss: 0.1671, EOP: 0.2071, EO: 0.2139, DP: 0.1167, Test Accuracy: 76.50%, Test EOP: 0.1195, Test EO: 0.1195, Test DP: 0.1186 Test AUROC: 0.8009


Training:  30%|███       | 3/10 [00:22<00:51,  7.37s/it]

...

Epoch [8/10], Loss: 0.4219, Fairness Loss: 0.1740, EOP: 0.2076, EO: 0.2156, DP: 0.1311, Test Accuracy: 80.18%, Test EOP: 0.1611, Test EO: 0.1611, Test DP: 0.1454 Test AUROC: 0.8341


Training:  90%|█████████ | 9/10 [01:06<00:07,  7.34s/it]

Epoch [9/10], Loss: 0.4258, Fairness Loss: 0.1697, EOP: 0.2076, EO: 0.2158, DP: 0.1216, Test Accuracy: 81.16%, Test EOP: 0.1474, Test EO: 0.1474, Test DP: 0.1314 Test AUROC: 0.8296


Training: 100%|██████████| 10/10 [01:13<00:00,  7.35s/it]

Epoch [10/10], Loss: 0.4253, Fairness Loss: 0.1703, EOP: 0.2123, EO: 0.2179, DP: 0.1178, Test Accuracy: 81.35%, Test EOP: 0.1432, Test EO: 0.1432, Test DP: 0.1171 Test AUROC: 0.8220


# 6) EOP:0.5 / OP:0 / DP:0.5

In [23]:
# 학습 함수
def train_model(model, train_loader, criterion, optimizer, device, lambda_fairness=0.01):
    model.train()
    total_loss = 0
    total_fairness_loss = 0
    total_eop = 0
    total_eo = 0
    total_dp = 0
    total_batches = 0

    for batch_x, batch_y, batch_sensitive in train_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        batch_sensitive = batch_sensitive.to(device)

        optimizer.zero_grad()

        # 예측 및 분류 손실
        outputs = model(batch_x)
        classification_loss = criterion(outputs, batch_y)

        # 공정성 손실 계산
        probabilities = torch.softmax(outputs, dim=1)[:, 1]  # Probability of class 1

        # To avoid NaNs, ensure that probabilities are between 0 and 1
        probabilities = torch.clamp(probabilities, min=0.0, max=1.0)

        # 공정성 지표 계산
        eop = FairnessMetrics.equal_opportunity_difference(probabilities, batch_y, batch_sensitive, target_label=1)
        eo = FairnessMetrics.equalized_odds_difference(probabilities, batch_y, batch_sensitive, target_label=1, epsilon=1e-6)
        dp = FairnessMetrics.demographic_parity_difference(probabilities, batch_sensitive)

        # 공정성 가중치 조정
        a = 0.5
        b = 0
        c = 0.5


        # 공정성 손실 합산
        fairness_loss = (a * eop) + (b * eo) +  (c * dp)

        # 전체 손실 결합
        total_loss_batch = classification_loss + lambda_fairness * fairness_loss
        total_loss_batch.backward()
        optimizer.step()

        # 손실 및 공정성 지표 기록
        total_loss += classification_loss.item()
        total_fairness_loss += fairness_loss.item()
        total_eop += eop.item()
        total_eo += eo.item()
        total_dp += dp.item()
        total_batches += 1

    avg_loss = total_loss / total_batches
    avg_fairness_loss = total_fairness_loss / total_batches
    avg_eop = total_eop / total_batches
    avg_eo = total_eo / total_batches
    avg_dp = total_dp / total_batches

    return avg_loss, avg_fairness_loss, avg_eop, avg_eo, avg_dp

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 모델 초기화
input_dim = X_train.shape[1]
hidden_dim = 128
num_heads = 4
num_classes = 2

model = FairnessAwareTransformerClassifier(input_dim, hidden_dim, num_heads, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10  # 충분한 학습을 위해 epoch 수 증가

# 기록을 위한 리스트 초기화
train_fairness_history = {
    'loss': [],
    'fairness_loss': [],
    'EOP': [],
    'EO': [],
    'DP': []
}
test_fairness_history = {
    'accuracy': [],
    'EOP': [],
    'EO': [],
    'DP': []
}

for epoch in tqdm(range(num_epochs), desc="Training"):
    train_loss, train_fairness_loss, train_eop, train_eo, train_dp = train_model(
        model, train_loader, criterion, optimizer,
        device, lambda_fairness=0.1
    )
    test_acc, test_eop, test_eo, test_dp, test_auroc = evaluate_model(model, test_loader, device)

    # 기록 저장
    train_fairness_history['loss'].append(train_loss)
    train_fairness_history['fairness_loss'].append(train_fairness_loss)
    train_fairness_history['EOP'].append(train_eop)
    train_fairness_history['EO'].append(train_eo)
    train_fairness_history['DP'].append(train_dp)

    test_fairness_history['accuracy'].append(test_acc)
    test_fairness_history['EOP'].append(test_eop)
    test_fairness_history['EO'].append(test_eo)
    test_fairness_history['DP'].append(test_dp)

    print(f'Epoch [{epoch+1}/{num_epochs}], '
          f'Loss: {train_loss:.4f}, '
          f'Fairness Loss: {train_fairness_loss:.4f}, '
          f'EOP: {train_eop:.4f}, '
          f'EO: {train_eo:.4f}, '
          f'DP: {train_dp:.4f}, '
          f'Test Accuracy: {test_acc:.2f}%, '
          f'Test EOP: {test_eop:.4f}, '
          f'Test EO: {test_eo:.4f}, '
          f'Test DP: {test_dp:.4f}',
          f'Test AUROC: {test_auroc:.4f}')

Training:  10%|█         | 1/10 [00:07<01:08,  7.56s/it]

Epoch [1/10], Loss: 0.4263, Fairness Loss: 0.1625, EOP: 0.2059, EO: 0.2133, DP: 0.1192, Test Accuracy: 82.65%, Test EOP: 0.1474, Test EO: 0.1474, Test DP: 0.1328 Test AUROC: 0.8554


Training:  20%|██        | 2/10 [00:14<00:57,  7.17s/it]

Epoch [2/10], Loss: 0.3805, Fairness Loss: 0.1847, EOP: 0.2266, EO: 0.2363, DP: 0.1429, Test Accuracy: 83.69%, Test EOP: 0.1231, Test EO: 0.1231, Test DP: 0.1497 Test AUROC: 0.8783


Training:  30%|███       | 3/10 [00:21<00:51,  7.34s/it]

...

Epoch [8/10], Loss: 0.3507, Fairness Loss: 0.1924, EOP: 0.2255, EO: 0.2379, DP: 0.1593, Test Accuracy: 84.32%, Test EOP: 0.0682, Test EO: 0.0856, Test DP: 0.1574 Test AUROC: 0.8865


Training:  90%|█████████ | 9/10 [01:05<00:07,  7.21s/it]

Epoch [9/10], Loss: 0.3509, Fairness Loss: 0.1764, EOP: 0.2038, EO: 0.2207, DP: 0.1490, Test Accuracy: 84.01%, Test EOP: 0.0650, Test EO: 0.0835, Test DP: 0.1475 Test AUROC: 0.8876


Training: 100%|██████████| 10/10 [01:12<00:00,  7.29s/it]

Epoch [10/10], Loss: 0.3473, Fairness Loss: 0.1896, EOP: 0.2137, EO: 0.2298, DP: 0.1655, Test Accuracy: 83.83%, Test EOP: 0.0719, Test EO: 0.0762, Test DP: 0.1425 Test AUROC: 0.8883


# 7) EOP:0.45 / OP:0.45 / DP: 0.1

In [24]:
# 학습 함수
def train_model(model, train_loader, criterion, optimizer, device, lambda_fairness=0.01):
    model.train()
    total_loss = 0
    total_fairness_loss = 0
    total_eop = 0
    total_eo = 0
    total_dp = 0
    total_batches = 0

    for batch_x, batch_y, batch_sensitive in train_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        batch_sensitive = batch_sensitive.to(device)

        optimizer.zero_grad()

        # 예측 및 분류 손실
        outputs = model(batch_x)
        classification_loss = criterion(outputs, batch_y)

        # 공정성 손실 계산
        probabilities = torch.softmax(outputs, dim=1)[:, 1]  # Probability of class 1

        # To avoid NaNs, ensure that probabilities are between 0 and 1
        probabilities = torch.clamp(probabilities, min=0.0, max=1.0)

        # 공정성 지표 계산
        eop = FairnessMetrics.equal_opportunity_difference(probabilities, batch_y, batch_sensitive, target_label=1)
        eo = FairnessMetrics.equalized_odds_difference(probabilities, batch_y, batch_sensitive, target_label=1, epsilon=1e-6)
        dp = FairnessMetrics.demographic_parity_difference(probabilities, batch_sensitive)

        # 공정성 가중치 조정
        a = 0.45
        b = 0.45
        c = 0.1


        # 공정성 손실 합산
        fairness_loss = (a * eop) + (b * eo) +  (c * dp)

        # 전체 손실 결합
        total_loss_batch = classification_loss + lambda_fairness * fairness_loss
        total_loss_batch.backward()
        optimizer.step()

        # 손실 및 공정성 지표 기록
        total_loss += classification_loss.item()
        total_fairness_loss += fairness_loss.item()
        total_eop += eop.item()
        total_eo += eo.item()
        total_dp += dp.item()
        total_batches += 1

    avg_loss = total_loss / total_batches
    avg_fairness_loss = total_fairness_loss / total_batches
    avg_eop = total_eop / total_batches
    avg_eo = total_eo / total_batches
    avg_dp = total_dp / total_batches

    return avg_loss, avg_fairness_loss, avg_eop, avg_eo, avg_dp

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 모델 초기화
input_dim = X_train.shape[1]
hidden_dim = 128
num_heads = 4
num_classes = 2

model = FairnessAwareTransformerClassifier(input_dim, hidden_dim, num_heads, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10  # 충분한 학습을 위해 epoch 수 증가

# 기록을 위한 리스트 초기화
train_fairness_history = {
    'loss': [],
    'fairness_loss': [],
    'EOP': [],
    'EO': [],
    'DP': []
}
test_fairness_history = {
    'accuracy': [],
    'EOP': [],
    'EO': [],
    'DP': []
}

for epoch in tqdm(range(num_epochs), desc="Training"):
    train_loss, train_fairness_loss, train_eop, train_eo, train_dp = train_model(
        model, train_loader, criterion, optimizer,
        device, lambda_fairness=0.1
    )
    test_acc, test_eop, test_eo, test_dp, test_auroc = evaluate_model(model, test_loader, device)

    # 기록 저장
    train_fairness_history['loss'].append(train_loss)
    train_fairness_history['fairness_loss'].append(train_fairness_loss)
    train_fairness_history['EOP'].append(train_eop)
    train_fairness_history['EO'].append(train_eo)
    train_fairness_history['DP'].append(train_dp)

    test_fairness_history['accuracy'].append(test_acc)
    test_fairness_history['EOP'].append(test_eop)
    test_fairness_history['EO'].append(test_eo)
    test_fairness_history['DP'].append(test_dp)

    print(f'Epoch [{epoch+1}/{num_epochs}], '
          f'Loss: {train_loss:.4f}, '
          f'Fairness Loss: {train_fairness_loss:.4f}, '
          f'EOP: {train_eop:.4f}, '
          f'EO: {train_eo:.4f}, '
          f'DP: {train_dp:.4f}, '
          f'Test Accuracy: {test_acc:.2f}%, '
          f'Test EOP: {test_eop:.4f}, '
          f'Test EO: {test_eo:.4f}, '
          f'Test DP: {test_dp:.4f}',
          f'Test AUROC: {test_auroc:.4f}')

Training:  10%|█         | 1/10 [00:07<01:03,  7.03s/it]

Epoch [1/10], Loss: 0.4451, Fairness Loss: 0.1592, EOP: 0.1638, EO: 0.1699, DP: 0.0902, Test Accuracy: 80.51%, Test EOP: 0.1087, Test EO: 0.1087, Test DP: 0.1099 Test AUROC: 0.8470


Training:  20%|██        | 2/10 [00:14<00:58,  7.26s/it]

Epoch [2/10], Loss: 0.3978, Fairness Loss: 0.1977, EOP: 0.1997, EO: 0.2108, DP: 0.1296, Test Accuracy: 82.94%, Test EOP: 0.0765, Test EO: 0.0765, Test DP: 0.1171 Test AUROC: 0.8667


Training:  30%|███       | 3/10 [00:21<00:51,  7.38s/it]

...

Epoch [8/10], Loss: 0.3562, Fairness Loss: 0.2042, EOP: 0.2005, EO: 0.2188, DP: 0.1556, Test Accuracy: 84.56%, Test EOP: 0.0409, Test EO: 0.0858, Test DP: 0.1448 Test AUROC: 0.8910


Training:  90%|█████████ | 9/10 [01:05<00:07,  7.29s/it]

Epoch [9/10], Loss: 0.3543, Fairness Loss: 0.2079, EOP: 0.2075, EO: 0.2223, DP: 0.1453, Test Accuracy: 84.61%, Test EOP: 0.0324, Test EO: 0.0732, Test DP: 0.1378 Test AUROC: 0.8798


Training: 100%|██████████| 10/10 [01:12<00:00,  7.29s/it]

Epoch [10/10], Loss: 0.3610, Fairness Loss: 0.1868, EOP: 0.1845, EO: 0.2000, DP: 0.1376, Test Accuracy: 83.66%, Test EOP: 0.0543, Test EO: 0.0989, Test DP: 0.1639 Test AUROC: 0.8866


# 8) EOP:0.25 / OP:0.25 / DP: 0.5

In [25]:
# 학습 함수
def train_model(model, train_loader, criterion, optimizer, device, lambda_fairness=0.01):
    model.train()
    total_loss = 0
    total_fairness_loss = 0
    total_eop = 0
    total_eo = 0
    total_dp = 0
    total_batches = 0

    for batch_x, batch_y, batch_sensitive in train_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        batch_sensitive = batch_sensitive.to(device)

        optimizer.zero_grad()

        # 예측 및 분류 손실
        outputs = model(batch_x)
        classification_loss = criterion(outputs, batch_y)

        # 공정성 손실 계산
        probabilities = torch.softmax(outputs, dim=1)[:, 1]  # Probability of class 1

        # To avoid NaNs, ensure that probabilities are between 0 and 1
        probabilities = torch.clamp(probabilities, min=0.0, max=1.0)

        # 공정성 지표 계산
        eop = FairnessMetrics.equal_opportunity_difference(probabilities, batch_y, batch_sensitive, target_label=1)
        eo = FairnessMetrics.equalized_odds_difference(probabilities, batch_y, batch_sensitive, target_label=1, epsilon=1e-6)
        dp = FairnessMetrics.demographic_parity_difference(probabilities, batch_sensitive)

        # 공정성 가중치 조정
        a = 0.25
        b = 0.25
        c = 0.5


        # 공정성 손실 합산
        fairness_loss = (a * eop) + (b * eo) +  (c * dp)

        # 전체 손실 결합
        total_loss_batch = classification_loss + lambda_fairness * fairness_loss
        total_loss_batch.backward()
        optimizer.step()

        # 손실 및 공정성 지표 기록
        total_loss += classification_loss.item()
        total_fairness_loss += fairness_loss.item()
        total_eop += eop.item()
        total_eo += eo.item()
        total_dp += dp.item()
        total_batches += 1

    avg_loss = total_loss / total_batches
    avg_fairness_loss = total_fairness_loss / total_batches
    avg_eop = total_eop / total_batches
    avg_eo = total_eo / total_batches
    avg_dp = total_dp / total_batches

    return avg_loss, avg_fairness_loss, avg_eop, avg_eo, avg_dp

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 모델 초기화
input_dim = X_train.shape[1]
hidden_dim = 128
num_heads = 4
num_classes = 2

model = FairnessAwareTransformerClassifier(input_dim, hidden_dim, num_heads, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10  # 충분한 학습을 위해 epoch 수 증가

# 기록을 위한 리스트 초기화
train_fairness_history = {
    'loss': [],
    'fairness_loss': [],
    'EOP': [],
    'EO': [],
    'DP': []
}
test_fairness_history = {
    'accuracy': [],
    'EOP': [],
    'EO': [],
    'DP': []
}

for epoch in tqdm(range(num_epochs), desc="Training"):
    train_loss, train_fairness_loss, train_eop, train_eo, train_dp = train_model(
        model, train_loader, criterion, optimizer,
        device, lambda_fairness=0.1
    )
    test_acc, test_eop, test_eo, test_dp, test_auroc = evaluate_model(model, test_loader, device)

    # 기록 저장
    train_fairness_history['loss'].append(train_loss)
    train_fairness_history['fairness_loss'].append(train_fairness_loss)
    train_fairness_history['EOP'].append(train_eop)
    train_fairness_history['EO'].append(train_eo)
    train_fairness_history['DP'].append(train_dp)

    test_fairness_history['accuracy'].append(test_acc)
    test_fairness_history['EOP'].append(test_eop)
    test_fairness_history['EO'].append(test_eo)
    test_fairness_history['DP'].append(test_dp)

    print(f'Epoch [{epoch+1}/{num_epochs}], '
          f'Loss: {train_loss:.4f}, '
          f'Fairness Loss: {train_fairness_loss:.4f}, '
          f'EOP: {train_eop:.4f}, '
          f'EO: {train_eo:.4f}, '
          f'DP: {train_dp:.4f}, '
          f'Test Accuracy: {test_acc:.2f}%, '
          f'Test EOP: {test_eop:.4f}, '
          f'Test EO: {test_eo:.4f}, '
          f'Test DP: {test_dp:.4f}',
          f'Test AUROC: {test_auroc:.4f}')

Training:  10%|█         | 1/10 [00:06<01:02,  6.91s/it]

Epoch [1/10], Loss: 0.4402, Fairness Loss: 0.1366, EOP: 0.1758, EO: 0.1828, DP: 0.0939, Test Accuracy: 82.08%, Test EOP: 0.0951, Test EO: 0.0951, Test DP: 0.1202 Test AUROC: 0.8513


Training:  20%|██        | 2/10 [00:14<00:58,  7.33s/it]

Epoch [2/10], Loss: 0.3849, Fairness Loss: 0.1670, EOP: 0.2033, EO: 0.2117, DP: 0.1266, Test Accuracy: 83.17%, Test EOP: 0.0830, Test EO: 0.0830, Test DP: 0.1240 Test AUROC: 0.8731


Training:  30%|███       | 3/10 [00:21<00:49,  7.13s/it]

...

Epoch [8/10], Loss: 0.3504, Fairness Loss: 0.1806, EOP: 0.2091, EO: 0.2225, DP: 0.1454, Test Accuracy: 84.01%, Test EOP: 0.0537, Test EO: 0.0885, Test DP: 0.1524 Test AUROC: 0.8850


Training:  90%|█████████ | 9/10 [01:05<00:07,  7.39s/it]

Epoch [9/10], Loss: 0.3506, Fairness Loss: 0.1953, EOP: 0.2230, EO: 0.2379, DP: 0.1602, Test Accuracy: 83.72%, Test EOP: 0.0641, Test EO: 0.0850, Test DP: 0.1512 Test AUROC: 0.8864


Training: 100%|██████████| 10/10 [01:12<00:00,  7.28s/it]

Epoch [10/10], Loss: 0.3481, Fairness Loss: 0.1810, EOP: 0.2048, EO: 0.2204, DP: 0.1494, Test Accuracy: 84.10%, Test EOP: 0.0292, Test EO: 0.0822, Test DP: 0.1470 Test AUROC: 0.8875


# 9) EOP:0.05 / OP:0.05 / DP: 0.9

In [26]:
# 학습 함수
def train_model(model, train_loader, criterion, optimizer, device, lambda_fairness=0.01):
    model.train()
    total_loss = 0
    total_fairness_loss = 0
    total_eop = 0
    total_eo = 0
    total_dp = 0
    total_batches = 0

    for batch_x, batch_y, batch_sensitive in train_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        batch_sensitive = batch_sensitive.to(device)

        optimizer.zero_grad()

        # 예측 및 분류 손실
        outputs = model(batch_x)
        classification_loss = criterion(outputs, batch_y)

        # 공정성 손실 계산
        probabilities = torch.softmax(outputs, dim=1)[:, 1]  # Probability of class 1

        # To avoid NaNs, ensure that probabilities are between 0 and 1
        probabilities = torch.clamp(probabilities, min=0.0, max=1.0)

        # 공정성 지표 계산
        eop = FairnessMetrics.equal_opportunity_difference(probabilities, batch_y, batch_sensitive, target_label=1)
        eo = FairnessMetrics.equalized_odds_difference(probabilities, batch_y, batch_sensitive, target_label=1, epsilon=1e-6)
        dp = FairnessMetrics.demographic_parity_difference(probabilities, batch_sensitive)

        # 공정성 가중치 조정
        a = 0.05
        b = 0.05
        c = 0.9


        # 공정성 손실 합산
        fairness_loss = (a * eop) + (b * eo) +  (c * dp)

        # 전체 손실 결합
        total_loss_batch = classification_loss + lambda_fairness * fairness_loss
        total_loss_batch.backward()
        optimizer.step()

        # 손실 및 공정성 지표 기록
        total_loss += classification_loss.item()
        total_fairness_loss += fairness_loss.item()
        total_eop += eop.item()
        total_eo += eo.item()
        total_dp += dp.item()
        total_batches += 1

    avg_loss = total_loss / total_batches
    avg_fairness_loss = total_fairness_loss / total_batches
    avg_eop = total_eop / total_batches
    avg_eo = total_eo / total_batches
    avg_dp = total_dp / total_batches

    return avg_loss, avg_fairness_loss, avg_eop, avg_eo, avg_dp

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 모델 초기화
input_dim = X_train.shape[1]
hidden_dim = 128
num_heads = 4
num_classes = 2

model = FairnessAwareTransformerClassifier(input_dim, hidden_dim, num_heads, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10  # 충분한 학습을 위해 epoch 수 증가

# 기록을 위한 리스트 초기화
train_fairness_history = {
    'loss': [],
    'fairness_loss': [],
    'EOP': [],
    'EO': [],
    'DP': []
}
test_fairness_history = {
    'accuracy': [],
    'EOP': [],
    'EO': [],
    'DP': []
}

for epoch in tqdm(range(num_epochs), desc="Training"):
    train_loss, train_fairness_loss, train_eop, train_eo, train_dp = train_model(
        model, train_loader, criterion, optimizer,
        device, lambda_fairness=0.1
    )
    test_acc, test_eop, test_eo, test_dp, test_auroc = evaluate_model(model, test_loader, device)

    # 기록 저장
    train_fairness_history['loss'].append(train_loss)
    train_fairness_history['fairness_loss'].append(train_fairness_loss)
    train_fairness_history['EOP'].append(train_eop)
    train_fairness_history['EO'].append(train_eo)
    train_fairness_history['DP'].append(train_dp)

    test_fairness_history['accuracy'].append(test_acc)
    test_fairness_history['EOP'].append(test_eop)
    test_fairness_history['EO'].append(test_eo)
    test_fairness_history['DP'].append(test_dp)

    print(f'Epoch [{epoch+1}/{num_epochs}], '
          f'Loss: {train_loss:.4f}, '
          f'Fairness Loss: {train_fairness_loss:.4f}, '
          f'EOP: {train_eop:.4f}, '
          f'EO: {train_eo:.4f}, '
          f'DP: {train_dp:.4f}, '
          f'Test Accuracy: {test_acc:.2f}%, '
          f'Test EOP: {test_eop:.4f}, '
          f'Test EO: {test_eo:.4f}, '
          f'Test DP: {test_dp:.4f}',
          f'Test AUROC: {test_auroc:.4f}')

Training:  10%|█         | 1/10 [00:07<01:08,  7.58s/it]

Epoch [1/10], Loss: 0.4510, Fairness Loss: 0.1089, EOP: 0.1755, EO: 0.1836, DP: 0.1011, Test Accuracy: 76.25%, Test EOP: 0.1083, Test EO: 0.1083, Test DP: 0.1093 Test AUROC: 0.7830


Training:  20%|██        | 2/10 [00:14<00:59,  7.48s/it]

Epoch [2/10], Loss: 0.4258, Fairness Loss: 0.1262, EOP: 0.2176, EO: 0.2241, DP: 0.1157, Test Accuracy: 80.15%, Test EOP: 0.1531, Test EO: 0.1531, Test DP: 0.1362 Test AUROC: 0.8346


Training:  30%|███       | 3/10 [00:22<00:50,  7.28s/it]

...

Epoch [8/10], Loss: 0.5160, Fairness Loss: 0.0441, EOP: 0.0886, EO: 0.0918, DP: 0.0389, Test Accuracy: 81.28%, Test EOP: 0.0981, Test EO: 0.0981, Test DP: 0.0827 Test AUROC: 0.8052


Training:  90%|█████████ | 9/10 [01:06<00:07,  7.41s/it]

Epoch [9/10], Loss: 0.4522, Fairness Loss: 0.0888, EOP: 0.1842, EO: 0.1872, DP: 0.0780, Test Accuracy: 80.94%, Test EOP: 0.0885, Test EO: 0.0885, Test DP: 0.0830 Test AUROC: 0.7834


Training: 100%|██████████| 10/10 [01:13<00:00,  7.32s/it]

Epoch [10/10], Loss: 0.4580, Fairness Loss: 0.0850, EOP: 0.1849, EO: 0.1881, DP: 0.0738, Test Accuracy: 79.30%, Test EOP: 0.0508, Test EO: 0.0508, Test DP: 0.0523 Test AUROC: 0.7360


10) EOP:0.25 / OP:0.5 / DP: 0.25

In [27]:
# 학습 함수
def train_model(model, train_loader, criterion, optimizer, device, lambda_fairness=0.01):
    model.train()
    total_loss = 0
    total_fairness_loss = 0
    total_eop = 0
    total_eo = 0
    total_dp = 0
    total_batches = 0

    for batch_x, batch_y, batch_sensitive in train_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        batch_sensitive = batch_sensitive.to(device)

        optimizer.zero_grad()

        # 예측 및 분류 손실
        outputs = model(batch_x)
        classification_loss = criterion(outputs, batch_y)

        # 공정성 손실 계산
        probabilities = torch.softmax(outputs, dim=1)[:, 1]  # Probability of class 1

        # To avoid NaNs, ensure that probabilities are between 0 and 1
        probabilities = torch.clamp(probabilities, min=0.0, max=1.0)

        # 공정성 지표 계산
        eop = FairnessMetrics.equal_opportunity_difference(probabilities, batch_y, batch_sensitive, target_label=1)
        eo = FairnessMetrics.equalized_odds_difference(probabilities, batch_y, batch_sensitive, target_label=1, epsilon=1e-6)
        dp = FairnessMetrics.demographic_parity_difference(probabilities, batch_sensitive)

        # 공정성 가중치 조정
        a = 0.25
        b = 0.5
        c = 0.25


        # 공정성 손실 합산
        fairness_loss = (a * eop) + (b * eo) +  (c * dp)

        # 전체 손실 결합
        total_loss_batch = classification_loss + lambda_fairness * fairness_loss
        total_loss_batch.backward()
        optimizer.step()

        # 손실 및 공정성 지표 기록
        total_loss += classification_loss.item()
        total_fairness_loss += fairness_loss.item()
        total_eop += eop.item()
        total_eo += eo.item()
        total_dp += dp.item()
        total_batches += 1

    avg_loss = total_loss / total_batches
    avg_fairness_loss = total_fairness_loss / total_batches
    avg_eop = total_eop / total_batches
    avg_eo = total_eo / total_batches
    avg_dp = total_dp / total_batches

    return avg_loss, avg_fairness_loss, avg_eop, avg_eo, avg_dp

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 모델 초기화
input_dim = X_train.shape[1]
hidden_dim = 128
num_heads = 4
num_classes = 2

model = FairnessAwareTransformerClassifier(input_dim, hidden_dim, num_heads, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10  # 충분한 학습을 위해 epoch 수 증가

# 기록을 위한 리스트 초기화
train_fairness_history = {
    'loss': [],
    'fairness_loss': [],
    'EOP': [],
    'EO': [],
    'DP': []
}
test_fairness_history = {
    'accuracy': [],
    'EOP': [],
    'EO': [],
    'DP': []
}

for epoch in tqdm(range(num_epochs), desc="Training"):
    train_loss, train_fairness_loss, train_eop, train_eo, train_dp = train_model(
        model, train_loader, criterion, optimizer,
        device, lambda_fairness=0.1
    )
    test_acc, test_eop, test_eo, test_dp, test_auroc = evaluate_model(model, test_loader, device)

    # 기록 저장
    train_fairness_history['loss'].append(train_loss)
    train_fairness_history['fairness_loss'].append(train_fairness_loss)
    train_fairness_history['EOP'].append(train_eop)
    train_fairness_history['EO'].append(train_eo)
    train_fairness_history['DP'].append(train_dp)

    test_fairness_history['accuracy'].append(test_acc)
    test_fairness_history['EOP'].append(test_eop)
    test_fairness_history['EO'].append(test_eo)
    test_fairness_history['DP'].append(test_dp)

    print(f'Epoch [{epoch+1}/{num_epochs}], '
          f'Loss: {train_loss:.4f}, '
          f'Fairness Loss: {train_fairness_loss:.4f}, '
          f'EOP: {train_eop:.4f}, '
          f'EO: {train_eo:.4f}, '
          f'DP: {train_dp:.4f}, '
          f'Test Accuracy: {test_acc:.2f}%, '
          f'Test EOP: {test_eop:.4f}, '
          f'Test EO: {test_eo:.4f}, '
          f'Test DP: {test_dp:.4f}',
          f'Test AUROC: {test_auroc:.4f}')

Training:  10%|█         | 1/10 [00:07<01:08,  7.64s/it]

Epoch [1/10], Loss: 0.4488, Fairness Loss: 0.1566, EOP: 0.1711, EO: 0.1787, DP: 0.0979, Test Accuracy: 81.52%, Test EOP: 0.1437, Test EO: 0.1437, Test DP: 0.1116 Test AUROC: 0.8451


Training:  20%|██        | 2/10 [00:14<00:57,  7.23s/it]

Epoch [2/10], Loss: 0.4043, Fairness Loss: 0.2022, EOP: 0.2190, EO: 0.2274, DP: 0.1351, Test Accuracy: 83.21%, Test EOP: 0.1607, Test EO: 0.1607, Test DP: 0.1554 Test AUROC: 0.8638


Training:  30%|███       | 3/10 [00:22<00:51,  7.41s/it]

...

Epoch [8/10], Loss: 0.3494, Fairness Loss: 0.2021, EOP: 0.2044, EO: 0.2224, DP: 0.1590, Test Accuracy: 84.23%, Test EOP: 0.0622, Test EO: 0.0972, Test DP: 0.1616 Test AUROC: 0.8881


Training:  90%|█████████ | 9/10 [01:05<00:07,  7.20s/it]

Epoch [9/10], Loss: 0.3474, Fairness Loss: 0.2007, EOP: 0.2039, EO: 0.2214, DP: 0.1562, Test Accuracy: 84.12%, Test EOP: 0.0378, Test EO: 0.0891, Test DP: 0.1481 Test AUROC: 0.8896


Training: 100%|██████████| 10/10 [01:13<00:00,  7.32s/it]

Epoch [10/10], Loss: 0.3492, Fairness Loss: 0.1988, EOP: 0.2016, EO: 0.2195, DP: 0.1545, Test Accuracy: 84.16%, Test EOP: 0.0525, Test EO: 0.1080, Test DP: 0.1754 Test AUROC: 0.8926
